# Packages install

In [2]:
# Install necessary packages
!pip install torch torchvision torchaudio
!pip install detectron2
!pip install clip
!pip install supabase

  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/51/9e/3b31f93a56d32f10db44bad3236a834c9e57fab66a9c6e5149ea666bb927/torchaudio-2.3.0-cp311-cp311-manylinux1_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.2 MB/s eta 0:00:0000:0100:01


# Imports 

In [3]:
import torch
from torchvision import transforms
import detectron2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
import clip
from PIL import Image
import os
import supabase


# Models load

In [ ]:
# Load CLIP model
def load_clip_model():
    model, preprocess = clip.load("ViT-B/32", device='cpu')
    return model, preprocess

# Load Detectron2 model
def load_detectron2_model():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x")
    predictor = DefaultPredictor(cfg)
    return predictor


# Interface essentials

In [7]:
import time
import requests
import numpy as np
import cv2
import torch
import clip
from PIL import Image
import io
import os
import supabase

# Konfiguracja globalnych zmiennych Supabase
SUPABASE_URL = "your_supabase_url"
SUPABASE_KEY = "your_supabase_key"

# Funkcja do pobierania obrazu z URL
def fetch_image_from_url(image_url):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = np.asarray(bytearray(response.content), dtype="uint8")
        im = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
        return im, image_data
    else:
        print(f"Failed to fetch image from {image_url}")
        return None, None

# Funkcja do przetwarzania obrazu za pomocą CLIP
def process_image_with_clip(image_data, model, preprocess):
    start_time = time.time()
    image = preprocess(Image.open(io.BytesIO(image_data))).unsqueeze(0).to('cpu')
    with torch.no_grad():
        image_features = model.encode_image(image)
    end_time = time.time()
    print(f"CLIP processing time: {round(end_time - start_time, 2)} seconds")
    return image_features

# Funkcja do przetwarzania obrazu za pomocą Detectron2
def process_detectron2(im):
    start_time = time.time()
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
    cfg.MODEL.DEVICE = "cpu"

    predictor = DefaultPredictor(cfg)
    outputs = predictor(im)

    # Wyodrębnienie klas wykrytych obiektów
    instances = outputs["instances"]
    classes = instances.pred_classes
    class_names = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
    detected_objects = [class_names[i] for i in classes]

    end_time = time.time()

    print("Detected objects:", detected_objects)
    print("Inference time:", round(end_time - start_time, 2), "seconds")
    return detected_objects

# Funkcja do zapisywania danych w Supabase
def save_to_supabase(image_url, clip_features):
    client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
    data = {
        "image_url": image_url,
        "clip_features": clip_features.tolist()
    }
    client.table("images").insert(data).execute()

# Funkcja do sprawdzania połączenia z Supabase
def check_supabase_connection():
    try:
        client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)
        response = client.table("images").select("*").limit(1).execute()
        if response.status_code == 200:
            print("Successfully connected to Supabase.")
        else:
            print("Failed to connect to Supabase.")
        return client
    except Exception as e:
        print(f"Error connecting to Supabase: {e}")
        return None

# Funkcja do inicjalizacji tabeli w Supabase
def initialize_supabase_table(client):
    try:
        response = client.table("images").select("*").execute()
        if response.status_code != 200:
            print("Initializing table 'images'.")
            client.sql("""
                CREATE TABLE IF NOT EXISTS images (
                    id UUID PRIMARY KEY,
                    image_url TEXT NOT NULL,
                    clip_features JSONB NOT NULL
                );
            """).execute()
        else:
            print("Table 'images' already exists.")
    except Exception as e:
        print(f"Error initializing table: {e}")

# Funkcja do ładowania modelu CLIP
def load_clip_model():
    model, preprocess = clip.load("ViT-B/32", device='cpu')
    return model, preprocess

# Funkcja do wyszukiwania obrazu
def search_photo():
    search_query = input("Enter a description of the photo you are looking for: ")
    # Implementacja logiki wyszukiwania obrazu w bazie danych
    pass

# Funkcja do przetwarzania lokalnych obrazów
def process_local_photos(path):
    clip_model, preprocess = load_clip_model()
    client = check_supabase_connection()
    initialize_supabase_table(client)
    for filename in os.listdir(path):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(path, filename)
            with open(image_path, "rb") as image_file:
                image_data = image_file.read()
            clip_start_time = time.time()
            clip_features = process_image_with_clip(image_data, clip_model, preprocess)
            clip_end_time = time.time()
            print(f"Total CLIP processing time for {filename}: {round(clip_end_time - clip_start_time, 2)} seconds")
            save_to_supabase(image_path, clip_features)

# Główne menu
def main_menu():
    clip_model, preprocess = load_clip_model()
    client = check_supabase_connection()
    if client:
        initialize_supabase_table(client)
    
    while True:
        print("\nSelect an option:")
        print("1. Upload photo by URL")
        print("2. Search a photo")
        print("3. Give local path to photos")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            image_url = input("Enter the URL of the image you want to upload: ")
            im, image_data = fetch_image_from_url(image_url)
            if im is not None:
                clip_start_time = time.time()
                clip_features = process_image_with_clip(image_data, clip_model, preprocess)
                clip_end_time = time.time()
                print(f"Total CLIP processing time: {round(clip_end_time - clip_start_time, 2)} seconds")
                
                save_to_supabase(image_url, clip_features)
                use_detectron = input("Do you want to use Detectron2 on this photo? (y/n): ")
                if use_detectron.lower() == 'y':
                    detectron2_start_time = time.time()
                    detected_objects = process_detectron2(im)
                    detectron2_end_time = time.time()
                    print(f"Total Detectron2 processing time: {round(detectron2_end_time - detectron2_start_time, 2)} seconds")
                    print("Detectron2 processing completed.")
        elif choice == '2':
            search_photo()
        elif choice == '3':
            path = input("Enter the local path to photos: ")
            process_local_photos(path)
        elif choice == '4':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please select again.")

if __name__ == "__main__":
    main_menu()


# Interface menu

In [8]:
def main_menu():
    clip_model, preprocess = load_clip_model()
    client = check_supabase_connection()
    if client:
        initialize_supabase_table(client)
    
    while True:
        print("\nSelect an option:")
        print("1. Upload photo by URL")
        print("2. Search a photo")
        print("3. Give local path to photos")
        print("4. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            image_url = input("Enter the URL of the image you want to upload: ")
            im, image_data = fetch_image_from_url(image_url)
            if im is not None:
                clip_start_time = time.time()
                clip_features = process_image_with_clip(image_data, clip_model, preprocess)
                clip_end_time = time.time()
                print(f"Total CLIP processing time: {round(clip_end_time - clip_start_time, 2)} seconds")
                
                save_to_supabase(image_url, clip_features)
                use_detectron = input("Do you want to use Detectron2 on this photo? (y/n): ")
                if use_detectron.lower() == 'y':
                    detectron2_start_time = time.time()
                    detected_objects = process_detectron2(im)
                    detectron2_end_time = time.time()
                    print(f"Total Detectron2 processing time: {round(detectron2_end_time - detectron2_start_time, 2)} seconds")
                    print("Detectron2 processing completed.")
        elif choice == '2':
            search_photo()
        elif choice == '3':
            path = input("Enter the local path to photos: ")
            process_local_photos(path)
        elif choice == '4':
            print("Exiting...")
            break
        else:
            print("Invalid choice. Please select again.")

if __name__ == "__main__":
    main_menu()


SupabaseException: Invalid URL